In [ ]:
import requests
import json
from pyspark.sql.functions import current_date, date_format

In [ ]:
df = spark.read.json("dbfs:/FileStore/tables/inbound/user_details/*.json")

df = df.withColumn("data_ref", date_format(current_date(), "yyyy-MM-dd"))

In [ ]:
df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("data_ref") \
    .save("dbfs:/FileStore/tables/bronze/user_details")